<a href="https://colab.research.google.com/github/bhkwon89/Section1_project/blob/main/cp2_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [68]:
# 모델 환경 tjfwjd
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="kykim/bert-kor-base", 
    downstream_corpus_name="emotion",
    downstream_model_dir="/content/drive/MyDrive/cp2/model",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=256,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7
)

In [69]:
# 랜덤시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [70]:
# 출력용 로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='kykim/bert-kor-base', downstream_task_name='document-classification', downstream_corpus_name='emotion', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/cp2/model', max_seq_length=256, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='kykim/bert-kor-base', downstream_task_name='document-classification', downstream_corpus_name='emotion', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/content/drive/MyDrive/cp2/model', max_seq_length=256, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_

In [87]:
import pandas as pd
train_df = pd.read_json('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Training.json')
val_df = pd.read_json('/content/drive/MyDrive/cp2/dataset/감성대화말뭉치(최종데이터)_Validation.json')

In [88]:
train_df.head()

,profile,talk
0,"{'persona-id': 'Pro_03719', 'persona': {'perso...","{'id': {'profile-id': 'Pro_03719', 'talk-id': ..."
1,"{'persona-id': 'Pro_10583', 'persona': {'perso...","{'id': {'profile-id': 'Pro_10583', 'talk-id': ..."
2,"{'persona-id': 'Pro_01781', 'persona': {'perso...","{'id': {'profile-id': 'Pro_01781', 'talk-id': ..."
3,"{'persona-id': 'Pro_10919', 'persona': {'perso...","{'id': {'profile-id': 'Pro_10919', 'talk-id': ..."
4,"{'persona-id': 'Pro_12418', 'persona': {'perso...","{'id': {'profile-id': 'Pro_12418', 'talk-id': ..."


In [89]:
situation_code = {
    1: '가족관계',
    2: '학업 및 진로',
    3: '학교폭력/따돌림',
    4: '대인관계',
    5: '연애,결혼,출산',
    6: '진로,취업,직장',
    7: '대인관계(부부,자녀)',
    8: '은퇴, 노후준비',
    9: '건강',
    10: '직작,업무,스트레스',
    11: '건강,죽음',
    12: '대인관계(노년)',
    13: '재정' 
}


emotion_code = {
    10: '분노',
    11: '툴툴대는',
    12: '좌절한',
    13: '짜증내는',
    14: '방어적인',
    15: '악의적인',
    16: '안달하는',
    17: '구역질나는',
    18: '노여워하는',
    19: '성가신',
    20: '슬픔',
    21: '실망한',
    22: '비통한',
    23: '후회되는',
    24: '우울한',
    25: '마비된',
    26: '염세적인',
    27: '눈물이 나는',
    28: '낙담한',
    29: '환멸을 느끼는',
    30: '불안',
    31: '두려운',
    32: '스트레스 받는',
    33: '취약한',
    34: '혼란스러운',
    35: '당혹스러운',
    36: '회의적인',
    37: '걱정스러운',
    38: '조심스러운',
    39: '초조한',
    40: '상처',
    41: '질투',
    42: '배신당한',
    43: '고립된',
    44: '충격 받은',
    45: '가난한, 불우한',
    46: '희생된',
    47: '억울한',
    48: '괴로워하는',
    49: '버려진',
    50: '당황',
    51: '고립된(당황한)',
    52: '남의 시선을 의식하는',
    53: '외로운',
    54: '열등감',
    55: '죄책감의',
    56: '부끄러운',
    57: '혐오스러운',
    58: '한심한',
    59: '혼란스러운',
    60: '기쁨',
    61: '감사하는',
    62: '신뢰하는',
    63: '편안한',
    64: '만족스러운',
    65: '흥분',
    66: '느긋',
    67: '안도',
    68: '신이 난',
    69: '자신하는'
}


In [90]:
df = pd.DataFrame(columns=['conversation', 'situation', 'emotion','basic_emotion'])
for i in range(len(train_df)):
  situation, disease, emotion =  dict(train_df.loc[i,'profile'])['emotion']['emotion-id'].split('_')
  conversation = (
    dict(train_df.loc[i,'talk'])['content']['HS01'] + ' ' + 
    dict(train_df.loc[i,'talk'])['content']['SS01'] + ' ' + 
    dict(train_df.loc[i,'talk'])['content']['HS02'] + ' ' + 
    dict(train_df.loc[i,'talk'])['content']['SS02'] + ' ' +
    dict(train_df.loc[i,'talk'])['content']['HS03'] + ' ' +
    dict(train_df.loc[i,'talk'])['content']['SS03'] )
  
  df_tmp = pd.DataFrame({
      'tesxt': [conversation],
      'situation' : [int(situation[1:])], 
      'emotion' : [int(emotion[1:])], 
      'label' : [int(emotion[1:])//10]
      })
  
  df = df.append(df_tmp)

In [91]:
df

,conversation,situation,emotion,basic_emotion,tesxt,label
0,NaN,5,68,NaN,아내가 드디어 출산하게 되어서 정말 신이 나. 아내분이 출산을 하시는군요. 정말 축...,6.0
0,NaN,11,32,NaN,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야. 약 종류가 ...,3.0
0,NaN,2,50,NaN,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워. 고등학교 수...,5.0
0,NaN,13,68,NaN,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. ...,6.0
0,NaN,13,67,NaN,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 기분 좋으시겠어요. 앞으로는 어...,6.0
...,...,...,...,...,...,...
0,NaN,4,34,NaN,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,3.0
0,NaN,9,67,NaN,지난주에 건강검진 결과가 나왔는데 정상이라고 결과가 나왔어. 검진 결과가 좋게 나와...,6.0
0,NaN,1,24,NaN,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼. 어머니의...,2.0
0,NaN,13,61,NaN,이렇게 좋은 운동 시설에서 경로 우대로 운동할 수 있다니 참 행운이야. 운동 시설을...,6.0


In [92]:
df.to_csv('/content/drive/MyDrive/cp2/emotion.csv')

In [93]:
from sklearn.model_selection import train_test_split

train_texts = df["text"].to_list()
train_labels = df["label"].to_list()

# Split Train and Validation data
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=0)

In [ ]:
pretrained_model_name = "kykim/bert-kor-base"

In [94]:
# 토크나이저 사용하기
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained( pretrained_model_name, do_lower_case=False)

train_encoded = tokenizer(train_texts, truncation=True, padding=True)
val_encoded = tokenizer(val_texts, truncation=True, padding=True)

NameError: ignored

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encoded),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encoded),
    val_labels
))


In [85]:
corpus = NsmcCorpus()
corpus.get_labels

<bound method NsmcCorpus.get_labels of <ratsnlp.nlpbook.classification.corpus.NsmcCorpus object at 0x7f87aec92ad0>>